## Setup

In [ ]:
import pandas as pd
import numpy as np
import folium
from ast import literal_eval
import re
import requests
from random import randint, shuffle

In [ ]:
dir = '/content/drive/My Drive/GSDS/'
user = pd.read_csv(dir+'user_id_map.csv')

In [ ]:
user.head()

,user_id_csv,user_id
0,0,8842281e1d1347389f2ab93d60773d4d
1,1,72fb0d0087d28c832f15776b0d936598
2,2,ab2923b738ea3082f5f3efcbbfacb218
3,3,d986f354a045ffb91234e4af4d1b12fd
4,4,7504b2aee1ecb5b2872d3da381c6c91e


In [ ]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876145 entries, 0 to 876144
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id_csv  876145 non-null  int64 
 1   user_id      876145 non-null  object
dtypes: int64(1), object(1)
memory usage: 13.4+ MB


## id, pw, email, nickname 생성

In [ ]:
#id unique한지 확인
user.shape[0], user['user_id_csv'].nunique(), user['user_id'].nunique()

(876145, 876145, 876145)

In [ ]:
# 닉네임 리스트 불러오기
url = 'https://svnweb.freebsd.org/csrg/share/dict/propernames?revision=61766&view=co'
r = requests.get(url)
names = r.text.split()
shuffle(names) #알파벳 순 -> random shuffling
print(len(names)) #1323개

1323


In [ ]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876145 entries, 0 to 876144
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id_csv  876145 non-null  int64 
 1   user_id      876145 non-null  object
 2   id           876145 non-null  int64 
 3   pw           876145 non-null  int64 
 4   email        8 non-null       object
dtypes: int64(3), object(2)
memory usage: 33.4+ MB


In [ ]:
user['id'] = user['user_id_csv']
user['pw'] = user['user_id_csv']
user['email'] = user['user_id'].apply(lambda x: x[:8] + '@bkms.com') #user_id 0-7번째 글자를 메일 아이디로!
user['nickname'] = user['id'].apply(lambda x: names[x % len(names)]+str(x))

In [ ]:
user.head()

,user_id_csv,user_id,id,pw,email,nickname
0,0,8842281e1d1347389f2ab93d60773d4d,0,0,8842281e@bkms.com,Aaron0
1,1,72fb0d0087d28c832f15776b0d936598,1,1,72fb0d00@bkms.com,Adam1
2,2,ab2923b738ea3082f5f3efcbbfacb218,2,2,ab2923b7@bkms.com,Adlai2
3,3,d986f354a045ffb91234e4af4d1b12fd,3,3,d986f354@bkms.com,Adrian3
4,4,7504b2aee1ecb5b2872d3da381c6c91e,4,4,7504b2ae@bkms.com,Agatha4


## 주소 데이터 전처리 및 확인

In [ ]:
'''
apt = pd.read_csv(dir+'seoul_latlong.csv', index_col=0)
def latlong_to_float(x):
  try:
    return float(x)
  except:
    return None
apt['위도'] = apt['위도'].apply(lambda x: latlong_to_float(x))
apt['경도'] = apt['경도'].apply(lambda x: latlong_to_float(x))
apt['address_gu'] = apt['주소'].apply(lambda x: x.split(' ')[1])
apt['address_dong'] = apt['주소'].apply(lambda x: x.split(' ')[2])
apt.columns = ['address', 'lat_long', 'lat', 'long', 'address_gu', 'address_dong']
apt = apt[['address', 'address_gu', 'address_dong', 'lat_long', 'lat', 'long']].dropna()
apt.to_csv(dir+'seoul_address.csv', index=False)
'''
# 구, 동 추출
# 알맞은 data type로 전환
# 위도, 경도 invalid 값 제거

"\napt = pd.read_csv(dir+'seoul_latlong.csv', index_col=0)\ndef latlong_to_float(x):\n  try:\n    return float(x)\n  except:\n    return None\napt['위도'] = apt['위도'].apply(lambda x: latlong_to_float(x))\napt['경도'] = apt['경도'].apply(lambda x: latlong_to_float(x))\napt['address_gu'] = apt['주소'].apply(lambda x: x.split(' ')[1])\napt['address_dong'] = apt['주소'].apply(lambda x: x.split(' ')[2])\napt.columns = ['address', 'lat_long', 'lat', 'long', 'address_gu', 'address_dong']\napt = apt[['address', 'address_gu', 'address_dong', 'lat_long', 'lat', 'long']].dropna()\napt.to_csv(dir+'seoul_address.csv', index=False)\n"

In [ ]:
apt = pd.read_csv(dir + 'seoul_address.csv', converters={'lat_long': pd.eval})
apt.head()

,address,address_gu,address_dong,lat_long,lat,long
0,서울특별시 금천구 시흥동 시흥성지,금천구,시흥동,"[37.4541515451134, 126.908853793235]",37.454152,126.908854
1,서울특별시 강남구 도곡동 도곡1차아이파크,강남구,도곡동,"[37.4922748332904, 127.043511837517]",37.492275,127.043512
2,서울특별시 양천구 신정동 신정대성유니드아파트2차,양천구,신정동,"[37.5210153975411, 126.848911008994]",37.521015,126.848911
3,서울특별시 노원구 중계동 중계라이프신동아청구아파트,노원구,중계동,"[37.6519317309728, 127.079543228581]",37.651932,127.079543
4,서울특별시 노원구 하계동 하계2차현대아파트,노원구,하계동,"[37.6446075819338, 127.072058758551]",37.644608,127.072059


In [ ]:
apt.info() #2746개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2746 entries, 0 to 2745
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   address       2746 non-null   object 
 1   address_gu    2746 non-null   object 
 2   address_dong  2746 non-null   object 
 3   lat_long      2746 non-null   object 
 4   lat           2746 non-null   float64
 5   long          2746 non-null   float64
dtypes: float64(2), object(4)
memory usage: 128.8+ KB


In [ ]:
apt[apt['address'].duplicated()]
apt[apt['address'] == apt.iloc[1630,0]] # 근데 위도, 경도는 다르니까 킵하겠음

,address,address_gu,address_dong,lat_long,lat,long
1164,서울특별시 양천구 목동 목동성원,양천구,목동,"[37.5358479839678, 126.869907138224]",37.535848,126.869907
1630,서울특별시 양천구 목동 목동성원,양천구,목동,"[37.5263981035576, 126.869331996718]",37.526398,126.869332


In [ ]:
m = folium.Map(location=apt.iloc[0,3],
               zoom_start=12, 
               width=750, 
               height=500)
folium.Marker(apt.iloc[0,3],
              tooltip=apt.iloc[0,0],
              popup = '0').add_to(m)
m

## 주소 랜덤 생성

In [ ]:
# lat, long에 랜덤 넣어서 320개 추가 생성
newdf = pd.DataFrame(np.repeat(apt.values, 320, axis=0))
newdf.columns = apt.columns
for i in range(newdf.shape[0]):
  if i%320!=0:
    newdf.iloc[i,4] = newdf.iloc[i,4] + np.random.normal(0,0.01)
    newdf.iloc[i,5] = newdf.iloc[i,5] + np.random.normal(0,0.01)
newdf['lat_long'] = newdf[['lat','long']].values.tolist()
newdf = newdf.sample(n = 876145)
newdf.head()

,address,address_gu,address_dong,lat_long,lat,long
202238,서울특별시 강서구 화곡동 화곡푸르지오,강서구,화곡동,"[37.5530912121662, 126.8453405996338]",37.553091,126.845341
212133,서울특별시 관악구 봉천동 관악드림타운,관악구,봉천동,"[37.4943283736794, 126.94497458628081]",37.494328,126.944975
174874,서울특별시 성동구 성수동1가 한양현대아파트,성동구,성수동1가,"[37.54149528543132, 127.04865040422466]",37.541495,127.04865
714398,서울특별시 양천구 신월동 신월시영아파트,양천구,신월동,"[37.51490090155186, 126.83286378019204]",37.514901,126.832864
864562,서울특별시 종로구 구기동 동익빌라,종로구,구기동,"[37.602795248660655, 126.95742088211823]",37.602795,126.957421


In [ ]:
newdf = newdf.reset_index(drop=True)
newdf.head()

,address,address_gu,address_dong,lat_long,lat,long
0,서울특별시 강서구 화곡동 화곡푸르지오,강서구,화곡동,"[37.5530912121662, 126.8453405996338]",37.553091,126.845341
1,서울특별시 관악구 봉천동 관악드림타운,관악구,봉천동,"[37.4943283736794, 126.94497458628081]",37.494328,126.944975
2,서울특별시 성동구 성수동1가 한양현대아파트,성동구,성수동1가,"[37.54149528543132, 127.04865040422466]",37.541495,127.04865
3,서울특별시 양천구 신월동 신월시영아파트,양천구,신월동,"[37.51490090155186, 126.83286378019204]",37.514901,126.832864
4,서울특별시 종로구 구기동 동익빌라,종로구,구기동,"[37.602795248660655, 126.95742088211823]",37.602795,126.957421


In [ ]:
newdf.shape, user.shape

((876145, 6), (876145, 6))

In [ ]:
newuser = pd.concat([user,newdf],axis=1)
newuser.head()

,user_id_csv,user_id,id,pw,email,nickname,address,address_gu,address_dong,lat_long,lat,long
0,0,8842281e1d1347389f2ab93d60773d4d,0,0,8842281e@bkms.com,Panos0,서울특별시 강서구 화곡동 화곡푸르지오,강서구,화곡동,"[37.5530912121662, 126.8453405996338]",37.553091,126.845341
1,1,72fb0d0087d28c832f15776b0d936598,1,1,72fb0d00@bkms.com,Leonard1,서울특별시 관악구 봉천동 관악드림타운,관악구,봉천동,"[37.4943283736794, 126.94497458628081]",37.494328,126.944975
2,2,ab2923b738ea3082f5f3efcbbfacb218,2,2,ab2923b7@bkms.com,Siping2,서울특별시 성동구 성수동1가 한양현대아파트,성동구,성수동1가,"[37.54149528543132, 127.04865040422466]",37.541495,127.04865
3,3,d986f354a045ffb91234e4af4d1b12fd,3,3,d986f354@bkms.com,Jarl3,서울특별시 양천구 신월동 신월시영아파트,양천구,신월동,"[37.51490090155186, 126.83286378019204]",37.514901,126.832864
4,4,7504b2aee1ecb5b2872d3da381c6c91e,4,4,7504b2ae@bkms.com,Hank4,서울특별시 종로구 구기동 동익빌라,종로구,구기동,"[37.602795248660655, 126.95742088211823]",37.602795,126.957421


In [ ]:
m = folium.Map(location=newuser.iloc[0,9],
               zoom_start=15, 
               width=750, 
               height=500)
folium.Marker(newuser.iloc[0,9],
              tooltip=newuser.iloc[0,5],
              popup = f'id: {newuser.iloc[0,0]}, nickname:{newuser.iloc[0,5]}, email: {newuser.iloc[0,4]}\naddress: {newuser.iloc[0,6]}').add_to(m)
m

In [ ]:
newuser.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876145 entries, 0 to 876144
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id_csv   876145 non-null  int64 
 1   user_id       876145 non-null  object
 2   id            876145 non-null  int64 
 3   pw            876145 non-null  int64 
 4   email         876145 non-null  object
 5   nickname      876145 non-null  object
 6   address       876145 non-null  object
 7   address_gu    876145 non-null  object
 8   address_dong  876145 non-null  object
 9   lat_long      876145 non-null  object
 10  lat           876145 non-null  object
 11  long          876145 non-null  object
dtypes: int64(3), object(9)
memory usage: 80.2+ MB


In [ ]:
newuser.to_parquet(dir+'user_info.parquet', index=False)

In [57]:
pd.read_parquet(dir+'user_info.parquet')

,user_id_csv,user_id,id,pw,email,nickname,address,address_gu,address_dong,lat_long,lat,long
0,0,8842281e1d1347389f2ab93d60773d4d,0,0,8842281e@bkms.com,Panos0,서울특별시 강서구 화곡동 화곡푸르지오,강서구,화곡동,"[37.5530912121662, 126.8453405996338]",37.553091,126.845341
1,1,72fb0d0087d28c832f15776b0d936598,1,1,72fb0d00@bkms.com,Leonard1,서울특별시 관악구 봉천동 관악드림타운,관악구,봉천동,"[37.4943283736794, 126.94497458628081]",37.494328,126.944975
2,2,ab2923b738ea3082f5f3efcbbfacb218,2,2,ab2923b7@bkms.com,Siping2,서울특별시 성동구 성수동1가 한양현대아파트,성동구,성수동1가,"[37.54149528543132, 127.04865040422466]",37.541495,127.048650
3,3,d986f354a045ffb91234e4af4d1b12fd,3,3,d986f354@bkms.com,Jarl3,서울특별시 양천구 신월동 신월시영아파트,양천구,신월동,"[37.51490090155186, 126.83286378019204]",37.514901,126.832864
4,4,7504b2aee1ecb5b2872d3da381c6c91e,4,4,7504b2ae@bkms.com,Hank4,서울특별시 종로구 구기동 동익빌라,종로구,구기동,"[37.602795248660655, 126.95742088211823]",37.602795,126.957421
...,...,...,...,...,...,...,...,...,...,...,...,...
876140,876140,d890e8079d8896e0cc6c4f178634850a,876140,876140,d890e807@bkms.com,Nils876140,서울특별시 중랑구 신내동 신내우남푸르미아,중랑구,신내동,"[37.59900791277032, 127.09750909620526]",37.599008,127.097509
876141,876141,71ca1aedba3bb07e9e914d072add7e48,876141,876141,71ca1aed@bkms.com,Carlos876141,서울특별시 중구 회현동1가 남산SK리더스뷰,중구,회현동1가,"[37.57742231697427, 126.98364122793403]",37.577422,126.983641
876142,876142,d0f6d1a4edcab80a6010cfcfeda4999f,876142,876142,d0f6d1a4@bkms.com,Stanislaw876142,서울특별시 중랑구 상봉동 신내건영1차,중랑구,상봉동,"[37.598615210557874, 127.09896541565891]",37.598615,127.098965
876143,876143,b0883ebf8e31731f1c5d91e678c26666,876143,876143,b0883ebf@bkms.com,Pim876143,서울특별시 강서구 방화동 마곡서광2차아파트,강서구,방화동,"[37.58086984627729, 126.8110043455924]",37.580870,126.811004
